<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-1/blob/main/BusquedaPorCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install seaborn==0.9.0

     |████████████████████████████████| 215kB 7.8MB/s 
  Found existing installation: seaborn 0.11.1
    Uninstalling seaborn-0.11.1:
      Successfully uninstalled seaborn-0.11.1


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
!wget https://www.dropbox.com/s/chfwr4x475y3p1v/noticiasFotos.zip?dl=0 -O 2.zip
!unzip 2.zip

In [5]:
!wget https://www.dropbox.com/s/lw24ve7ndobd4y5/EmbeddingsFotos.csv?dl=0 -O EmbeddingsFotos.csv

--2021-05-27 14:07:49--  https://www.dropbox.com/s/lw24ve7ndobd4y5/EmbeddingsFotos.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/lw24ve7ndobd4y5/EmbeddingsFotos.csv [following]
--2021-05-27 14:07:49--  https://www.dropbox.com/s/raw/lw24ve7ndobd4y5/EmbeddingsFotos.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf5eec89c0f9121582e94d4a91f.dl.dropboxusercontent.com/cd/0/inline/BPReIJ6PslwiIOKj6FgJREJ0-zlHdjQkEn_NxPXY9YhLo5qe9gLnhf-Is2IyzctkHVaWKyYdbo0LgLLg_g-sgoo8GZMjvjDernl0ERJ5jofRxprP9nX2_B5I7FzTelImhidMegvBhP1Lh_tRPC28ex-a/file# [following]
--2021-05-27 14:07:50--  https://ucf5eec89c0f9121582e94d4a91f.dl.dropboxusercontent.com/cd/0/inline/BPReIJ6PslwiIOKj6FgJREJ0-zlHdjQkEn_NxPXY9YhLo5qe9gLnhf-Is2Iyzctk

In [6]:
df = pd.read_csv('EmbeddingsFotos.csv')

In [7]:
df.head()

,NombreFoto,Embedding
0,462220.jpg 0,[[ 2.3611324 0.7700528 1.5315906 -1.38208...
1,462220.jpg 1,[[-0.18996963 0.47556552 0.3286136 1.53095...
2,44569.jpg 0,[[-0.67055297 -0.4772751 -0.05659456 0.97165...
3,44569.jpg 1,[[-8.7787104e-01 1.4074993e+00 -9.2881995e-01...
4,44569.jpg 2,[[ 0.55190855 0.47906682 -1.9257076 -0.58596...


Foto Entrada

In [9]:
!wget https://www.dropbox.com/s/2t7n2q8b8gidjsh/foto.zip?dl=0 -O foto.zip
!unzip foto.zip

--2021-05-27 14:11:51--  https://www.dropbox.com/s/2t7n2q8b8gidjsh/foto.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/2t7n2q8b8gidjsh/foto.zip [following]
--2021-05-27 14:11:51--  https://www.dropbox.com/s/raw/2t7n2q8b8gidjsh/foto.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc97f3e88d844af30efdb1709003.dl.dropboxusercontent.com/cd/0/inline/BPQnCW2sYwOpOiElwtb8ivE84jVovxzvrFJTWgd5j8H4hXx9LYG72z03cEQIL8mzjdxEQ60OMobSgN1W7ph2kWQ5Elg7Fez16T3BwLHXByRjSpIiC3McHrUWZ3FOzPkTs5avpFHKa6GW3pYecGwZ2L5S/file# [following]
--2021-05-27 14:11:52--  https://uc97f3e88d844af30efdb1709003.dl.dropboxusercontent.com/cd/0/inline/BPQnCW2sYwOpOiElwtb8ivE84jVovxzvrFJTWgd5j8H4hXx9LYG72z03cEQIL8mzjdxEQ60OMobSgN1W7ph2kWQ5Elg7Fez16

### Métodos necesarios

In [13]:
def load_image(DIR, NAME):
    return cv2.cvtColor(cv2.imread(f'{DIR}/{name}'), cv2.COLOR_BGR2RGB)

def detect_faces(image, score_threshold=0.7):
    global boxes, scores
    (imh, imw) = image.shape[:-1]
    img = np.expand_dims(image,axis=0)
    
    # Inicializar mobilenet
    sess = tf.compat.v1.Session(graph=mobilenet)
    image_tensor = mobilenet.get_tensor_by_name('image_tensor:0')
    boxes = mobilenet.get_tensor_by_name('detection_boxes:0')
    scores = mobilenet.get_tensor_by_name('detection_scores:0')
    
    # Predicción (detección)
    (boxes, scores) = sess.run([boxes, scores], feed_dict={image_tensor:img})
    
    # Reajustar tamaños boxes, scores
    boxes = np.squeeze(boxes,axis=0)
    scores = np.squeeze(scores,axis=0)
    
    # Depurar bounding boxes
    idx = np.where(scores>=score_threshold)[0]
    
    # Crear bounding boxes
    bboxes = []
    for index in idx:
        ymin, xmin, ymax, xmax = boxes[index,:]
        (left, right, top, bottom) = (xmin*imw, xmax*imw, ymin*imh, ymax*imh)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        bboxes.append([left,right,top,bottom])
        
    return bboxes


def extract_faces(image,bboxes,new_size=(160,160)):
    cropped_faces = []
    for box in bboxes:
        left, right, top, bottom = box
        face = image[top:bottom,left:right]
        cropped_faces.append(cv2.resize(face,dsize=new_size))
    return cropped_faces

In [14]:
known_embeddings = []

print('Procesando rostros conocidos...')
for name in os.listdir('foto'):
    if name.endswith('.jpg'):
        print(f'   {name}')
        image = load_image('foto',name)
        bboxes = detect_faces(image)
        face = extract_faces(image,bboxes)
        known_embeddings.append(compute_embedding(facenet,face[0]))

Procesando rostros conocidos...
   conchaAndreu.jpg


NameError: ignored